In [ ]:
# Set TensorFlow version to 1.x
%tensorflow_version 1.x

# Print assigned GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# IMPORTANT: Ensure that the repository directory follows the path (or update all paths in the script):
# "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

In [11]:
# Download dataset to pre-specified folder on local VM instance
%cd /content/

import time
from datetime import datetime, timedelta

start = time.time()

!bash "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose/scripts/coco_dl.sh" ./datasets

download_time = time.time() - start
print("Total download time: {}".format(str(timedelta(seconds=download_time))))

/content
train2017.zip and train2017/ not found!
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0G  100 18.0G    0     0  71.8M      0  0:04:16  0:04:16 --:--:-- 75.8M
Unzipping: train2017.zip
Deleting file: train2017.zip
val2017.zip and val2017/ not found!
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  777M  100  777M    0     0  55.9M      0  0:00:13  0:00:13 --:--:-- 55.8M
Unzipping: val2017.zip
Deleting file: val2017.zip
annotations_trainval2017.zip not found!
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241M  100  241M    0     0  70.1M      0  0:00:03  0:00:03 --:--:-- 70.1M
Unzipping: annotations_trainval2017.zip
Deleting file: annot

In [12]:
!ls /content/datasets/coco/train2017 | wc -l
!ls /content/datasets/coco/val2017 | wc -l

118287
5000


In [13]:
# Ensure that filesystem is set up correctly and will not be the bottleneck
import time
from datetime import datetime, timedelta

start = time.time()

import os
def print_files_in_dir(path):
    print(len(os.listdir(path)), " files at ", path)

print_files_in_dir('/content/datasets/coco/train2017/')
print_files_in_dir('/content/datasets/coco/val2017/')


setup_time = time.time() - start
print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

if setup_time > 1:
    print("There appears to be a bottleneck with filesystem loading time. This may severely impact training speed.")

118287  files at  /content/datasets/coco/train2017/
5000  files at  /content/datasets/coco/val2017/
Total setup time: 0:00:00.076181


In [14]:
# Change dir to project dir
%cd "/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose"

# Toggle COLAB_TRAINING variable in constants file
# https://askubuntu.com/questions/20414/find-and-replace-text-within-a-file-using-commands
!sed -i.bak 's/COLAB_TRAINING = False/COLAB_TRAINING = True/g' constants.py 

# Remove backup file
!rm constants.py.bak

/content/gdrive/MyDrive/Colab Data/COCO-Human-Pose


In [47]:
!head train.py
!google-drive-ocamlfuse -cc 
# refresh collab session with latest source files in GDrive

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

import argparse
import os
import time
from datetime import datetime, timedelta

import tensorflow as tf
from tensorflow.compat.v1.keras import backend as k

from constants import *
from hourglass import HourglassNet
/bin/bash: google-drive-ocamlfuse: command not found
Tensorflow version 2.4.1


In [48]:
!python3 train.py --epochs 100 --batch 128 --hourglass 4 --subset 1.0

# To resume, uncomment the following and point it to the subdirectory. Ensure hourglass matches.
# !python3 train.py --epochs 100 --batch 12 --hourglass 4 --subset 1.0 --resume True --resume-subdir "2021-03-13-16h-38m_batchsize_12_hg_4" --resume-epoch 9

# Note about subset: if you decide to run on a subset <1.0, there is no functionality to retrieve the same subset of data if the model is stopped then resumed. For this reason, subset should only be used for quick tests rather than reporting robust/repeatable results. 

2021-03-18 19:42:28.020046: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Setup start: Thu Mar 18 19:42:31 2021

TensorFlow detected the following GPU(s):
2021-03-18 19:42:31.040130: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-18 19:42:31.041268: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-18 19:42:31.052285: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-18 19:42:31.052339: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5b007bb24136): /proc/driver/nvidia/version does not exist

2021-03-18 19:42:31.053545: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices 